In [28]:
# Regression Example With Boston Dataset: Standardized 
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import scale
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn import preprocessing


#load data
path = 'C:/Users/mys12/Desktop/northeastern/summer2018/data_Robust_Fingerprinting-master/DISTRIBUTED_OPENSOURCE/FINGERPRINTING_DB'

train_rss = pd.read_csv(path + '/Training_rss_21Aug17.csv', header = 0)
train_coord = pd.read_csv(path + '/Training_coordinates_21Aug17.csv', header = 0)

test_rss = pd.read_csv(path + '/Test_rss_21Aug17.csv', header = 0)
test_coord = pd.read_csv(path + '/Test_coordinates_21Aug17.csv', header = 0)


In [29]:

# test
test_r = test_rss.values
test_ch = test_coord.ix[:,-1]
test_labels = np.asarray(test_ch.map(str))
print("test_labels: ", np.unique(test_labels, return_counts=True))

test_labels = np.asarray(pd.get_dummies(test_labels))

#Scale transforms data to center to the mean and component wise scale to unit variance
normalizer = preprocessing.Normalizer().fit(np.asarray(test_r))
test_r_features = normalizer.transform(np.asarray(test_r))
print(test_r_features.shape)

# train 
train_r = train_rss.values
train_ch = train_coord.iloc[:,-1]
building_floors_str = train_ch.map(str)  #convert all the building floors to strings
print("train_labels: ", np.unique(building_floors_str, return_counts=True))

train_labels = np.asarray(building_floors_str)
#convert labels to categorical variables, dummy_labels has type 'pandas.core.frame.DataFrame'
dummy_label = pd.get_dummies(train_labels)
train_labels = np.asarray(dummy_label)

train_r_features = normalizer.transform(np.asarray(train_r))
print(train_r_features.shape)


test_labels:  (array(['0.0', '11.1', '14.8', '3.7', '7.4'], dtype=object), array([1264,  699,  109, 1108,  770], dtype=int64))
(3950, 992)
train_labels:  (array(['0.0', '11.1', '14.8', '3.7', '7.4'], dtype=object), array([226, 118,  17, 197, 138], dtype=int64))
(696, 992)


C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  after removing the cwd from sys.path.
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\sklearn\utils\validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [30]:
#generate len(test_r_features) of floats in between 0 and 1
train_val_split = np.random.rand(len(test_r_features))
#convert train_val_split to an array of booleans: if elem < 0.7 = true, else: false
train_val_split = train_val_split < 0.70 #should contain ~70% percent true

In [31]:
# We will then split our given training set into training + validation 
# change the train to test, because the test has more data
train_X = test_r_features[train_val_split]  
train_y = test_labels[train_val_split]
val_X = test_r_features[~train_val_split]
val_y = test_labels[~train_val_split]

In [40]:
nb_epochs = 100
batch_size = 64
input_size = 992
num_classes = 5

In [41]:
def encoder():
    model = Sequential()
    model.add(Dense(256, input_dim=input_size, activation='tanh', bias=True))
    model.add(Dense(128, activation='tanh', bias=True))
    model.add(Dense(64, activation='tanh', bias=True))
    return model

In [42]:
def decoder(e):   
    e.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    e.add(Dense(256, activation='tanh', bias=True))
    e.add(Dense(input_size, activation='tanh', bias=True))
    e.compile(optimizer='adam', loss='mse')
    return e

In [43]:
e = encoder()
d = decoder(e)
d.fit(train_X, train_X, nb_epoch=nb_epochs, batch_size=batch_size)

C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, input_dim=992, activation="tanh", use_bias=True)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  after removing the cwd from sys.path.
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, activation="tanh", use_bias=True)`
  """
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=64, activation="tanh", use_bias=True)`
  
C:\software\WinPy

Epoch 1/100
2770/2770 [==============================] - 0s - loss: 3.7499e-04     
Epoch 2/100
2770/2770 [==============================] - 0s - loss: 1.2332e-04     
Epoch 3/100
2770/2770 [==============================] - 0s - loss: 9.5317e-05     
Epoch 4/100
2770/2770 [==============================] - 0s - loss: 8.2359e-05     
Epoch 5/100
2770/2770 [==============================] - 0s - loss: 7.3340e-05     
Epoch 6/100
2770/2770 [==============================] - 0s - loss: 6.5863e-05     
Epoch 7/100
2770/2770 [==============================] - 0s - loss: 6.0079e-05     
Epoch 8/100
2770/2770 [==============================] - 0s - loss: 5.5825e-05     
Epoch 9/100
2770/2770 [==============================] - 0s - loss: 5.2495e-05     
Epoch 10/100
2770/2770 [==============================] - 0s - loss: 4.9815e-05     
Epoch 11/100
2770/2770 [==============================] - 0s - loss: 4.7307e-05     
Epoch 12/100
2770/2770 [==============================] - 0s - loss: 4.524

2770/2770 [==============================] - 0s - loss: 2.3644e-05     
Epoch 98/100
2770/2770 [==============================] - 0s - loss: 2.3600e-05     
Epoch 99/100
2770/2770 [==============================] - 0s - loss: 2.3508e-05     
Epoch 100/100
2770/2770 [==============================] - 0s - loss: 2.3435e-05     


In [44]:
def classifier(d):
    num_to_remove = 3
    for i in range(num_to_remove):
        d.pop()
    d.add(Dense(128, input_dim=64, activation='tanh', bias=True))
    d.add(Dense(128, activation='tanh', bias=True))
    d.add(Dense(num_classes, activation='softmax', bias=True))
    d.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    return d

In [45]:
c = classifier(d)
c.fit(train_X, train_y, validation_data=(val_X, val_y), nb_epoch=nb_epochs, batch_size=batch_size)

C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=64, activation="tanh", use_bias=True)`
  """
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, activation="tanh", use_bias=True)`
  
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(5, activation="softmax", use_bias=True)`
  import sys
C:\software\WinPython-64bit-3.6.2.0Qt5\python-3.6.2.amd64\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 2770 samples, validate on 1180 samples
Epoch 1/100
2770/2770 [==============================] - 0s - loss: 1.0045 - acc: 0.5477 - val_loss: 0.7864 - val_acc: 0.7568
Epoch 2/100
2770/2770 [==============================] - 0s - loss: 0.5047 - acc: 0.8007 - val_loss: 0.3567 - val_acc: 0.8686
Epoch 3/100
2770/2770 [==============================] - 0s - loss: 0.3651 - acc: 0.8578 - val_loss: 0.2922 - val_acc: 0.9051
Epoch 4/100
2770/2770 [==============================] - 0s - loss: 0.3055 - acc: 0.8931 - val_loss: 0.2589 - val_acc: 0.9203
Epoch 5/100
2770/2770 [==============================] - 0s - loss: 0.2983 - acc: 0.8996 - val_loss: 0.4030 - val_acc: 0.8415
Epoch 6/100
2770/2770 [==============================] - 0s - loss: 0.3567 - acc: 0.8697 - val_loss: 0.3585 - val_acc: 0.8703
Epoch 7/100
2770/2770 [==============================] - 0s - loss: 0.2672 - acc: 0.9083 - val_loss: 0.4817 - val_acc: 0.8203
Epoch 8/100
2770/2770 [==============================] - 0s - loss: 0.

2770/2770 [==============================] - 0s - loss: 0.1934 - acc: 0.9278 - val_loss: 0.4666 - val_acc: 0.8695
Epoch 66/100
2770/2770 [==============================] - 0s - loss: 0.1665 - acc: 0.9433 - val_loss: 0.2609 - val_acc: 0.9246
Epoch 67/100
2770/2770 [==============================] - 0s - loss: 0.1500 - acc: 0.9477 - val_loss: 0.2344 - val_acc: 0.9288
Epoch 68/100
2770/2770 [==============================] - 0s - loss: 0.1259 - acc: 0.9560 - val_loss: 0.2086 - val_acc: 0.9441
Epoch 69/100
2770/2770 [==============================] - 0s - loss: 0.1113 - acc: 0.9610 - val_loss: 0.2205 - val_acc: 0.9415
Epoch 70/100
2770/2770 [==============================] - 0s - loss: 0.1178 - acc: 0.9570 - val_loss: 0.2350 - val_acc: 0.9271
Epoch 71/100
2770/2770 [==============================] - 0s - loss: 0.1282 - acc: 0.9484 - val_loss: 0.2257 - val_acc: 0.9305
Epoch 72/100
2770/2770 [==============================] - 0s - loss: 0.1361 - acc: 0.9527 - val_loss: 0.3745 - val_acc: 0.88

In [46]:
loss, acc = c.evaluate(train_r_features, train_labels)

576/696 [=======================>......] - ETA: 0s

In [47]:
print(loss, acc)

0.306821404004 0.913793102763
